In [ ]:
import tensorflow

In [ ]:
import numpy as np
import time
# from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

In [ ]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    # [3,4] after normalize => [3/5,4/5]

    # K.epsilon()) = 10^-7, prevent from dividing by zero
    
    return x / ( K.sqrt(K.mean(K.square(x))) + K.epsilon() )



In [ ]:
K.image_data_format()

In [ ]:
model = vgg16.VGG16(weights='imagenet', include_top=False)
model.summary()


In [ ]:
filter_index = 0
print('Processing filter %d' % filter_index)


layer_name = 'block5_conv1'
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


input_img = model.input
layer_output = layer_dict[layer_name].output

loss = K.mean(layer_output[:, :, :, filter_index])
loss

In [ ]:
filter_index = 0
print('Processing filter %d' % filter_index)


layer_name = 'block5_conv1'
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])


input_img = model.input
layer_output = layer_dict[layer_name].output

loss = K.mean(layer_output[:, :, :, filter_index])

# return a list of gradient tensor
grads = K.gradients(loss, [input_img])[0]
# grads = K.gradients(loss, input_img)[0]
grads = normalize(grads)

# function = K.function([input_img], [loss, grads])
function = K.function([input_img], [loss,grads,layer_output])


lr = 1.
img_width = 128
img_height = 128


# 128 +/- 10, 138 > value > 118
input_img_data = np.random.random((1, img_width, img_height, 3))
input_img_data = (input_img_data - 0.5) * 20 + 128

# we run gradient ascent for 20 steps
for i in range(100):
    
    print(' -------------------------------- %s --------------------------------' %i)
    
    loss_value, grads_value, layer_output = function([input_img_data])
#     loss_value, grads_value = function([input_img_data])

    print(grads_value.shape)
    print('input_img_data.shape',input_img_data.shape)
    print('layer_output.shape',layer_output.shape)
    print('type(layer_output)',type(layer_output),np.mean(layer_output[:, :, :, filter_index]))
    print('input_img_data[0,0,0:2]',input_img_data[0,0,0:2])
    print('grads_value[0,0,0:2]',grads_value[0,0,0:2])
    
    input_img_data = input_img_data + grads_value * lr
    
    print('_____after gradient acsent_____')
    print('input_img_data',input_img_data[0,0,0:2])
    

    print('Current loss value:', loss_value)
    
    if loss_value <= 0.:
        # some filters get stuck to 0, we can skip them
        break
        



In [ ]:
import matplotlib.pyplot as plt

img = deprocess_image(input_img_data[0])


plt.figure(figsize=(4, 20))
ax = plt.subplot(1, 1, 1)

plt.imshow(img)

ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

plt.show()